In [19]:
import cv2
import mediapipe as mp
import os
import numpy as np
import math
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils

# Step 1: Calculate Joint Angles
def calculate_angle(landmark1, landmark2, landmark3):
    x1, y1, _ = landmark1
    x2, y2, _ = landmark2
    x3, y3, _ = landmark3
    angle = math.degrees(math.atan2(y3 - y2, x3 - x2) - math.atan2(y1 - y2, x1 - x2))
    return angle if angle >= 0 else angle + 360

# Step 2: Extract Pose Features
def extract_pose_features(image):
    """
    Detect pose landmarks and extract joint angles as features.
    """
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = pose.process(image_rgb)
    if not results.pose_landmarks:
        return None  # No pose detected
    
    landmarks = results.pose_landmarks.landmark
    features = {
        'left_elbow_angle': calculate_angle(
            (landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
             landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y, 0),
            (landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
             landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y, 0),
            (landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
             landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y, 0)),
        'right_elbow_angle': calculate_angle(
            (landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
             landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y, 0),
            (landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,
             landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y, 0),
            (landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,
             landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y, 0)),
        'left_shoulder_angle': calculate_angle(
            (landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
             landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y, 0),
            (landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
             landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y, 0),
            (landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
             landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y, 0)),
        'right_shoulder_angle': calculate_angle(
            (landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,
             landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y, 0),
            (landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
             landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y, 0),
            (landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,
             landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y, 0))
    }
    return features

# Step 3: Process Dataset and Extract Features
def process_dataset(dataset_path):
    data = []
    for label in os.listdir(dataset_path):
        label_path = os.path.join(dataset_path, label)
        if not os.path.isdir(label_path):
            continue
        for image_name in os.listdir(label_path):
            image_path = os.path.join(label_path, image_name)
            image = cv2.imread(image_path)
            if image is None:
                continue
            features = extract_pose_features(image)
            if features:
                features['label'] = label
                data.append(features)
    return pd.DataFrame(data)

# Step 4: Train Classifier
def train_classifier(dataframe):
    X = dataframe.drop('label', axis=1)
    y = dataframe['label']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    clf = RandomForestClassifier()
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(f"Model Accuracy: {accuracy_score(y_test, y_pred):.2f}")
    return clf

# Step 5: Real-Time Gesture Recognition
def real_time_recognition(classifier):
    cap = cv2.VideoCapture(0)
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        features = extract_pose_features(frame)
        if features:
            features_array = np.array([list(features.values())])
            label = classifier.predict(features_array)[0]
            cv2.putText(frame, label, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        
        cv2.imshow("Gesture Recognition", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# Main Function
if __name__ == "__main__":
    dataset_path = 'Dataset'
    print("Processing dataset...")
    dataframe = process_dataset(dataset_path)
    print(f"Extracted features from {len(dataframe)} images.")

    print("Training classifier...")
    classifier = train_classifier(dataframe)

    print("Starting real-time gesture recognition...")
    real_time_recognition(classifier)


KeyboardInterrupt: 